#### EVALUATION FILE

##### This file is to evaluate the loaded models with an specific validation keys

In [1]:
import torch
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.transforms.functional import crop
import gc

from torch.utils.data import Dataset
from Utils.logger import initialize_logger, get_logger
import numpy as np
import torch
from scipy import signal
from Utils.config import (
    USE_PHYSICAL_DATA,
    PATH_DATASET,
    IMG_PATH,
)

from Utils import (
    metrics,
)
# Metrics
metrics = [
    torch.nn.MSELoss(),
    metrics.PerCS()
]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LOCAL_SLP_DATASET_PATH = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(__file__)))),'SLP/danaLab')
SERVER_SLP_DATASET_PATH = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(__file__)))))),'mnt/DADES2/SLP/SLP/danaLab')

List of the valuation patients:

['00074', '00040', '00024', '00043', '00081', '00070', '00077', '00044', '00002','00079', '00091', '00087', '00078', '00047', '00031', '00069', '00039', '00086', '00009', '00092', '00064']

In [ ]:
# Function to get the numpy arrays of an specific image

def select_file(path,patient,cover,img):

    if os.path.isdir(os.path.join(path,patient)):
        print('Patient:', patient)
        patient_path = os.path.join(path,patient)
        pm_np_path = os.path.join(patient_path,'PMarray')
        ir_np_path = os.path.join(patient_path,'IRraw')

        if os.path.isdir(pm_np_path) and os.path.isdir(ir_np_path):
            pm_cover_path = os.path.join(pm_np_path,cover)
            print('PM cover path:',pm_cover_path)
            ir_cover_path = os.path.join(ir_np_path,cover)
            print('IR cover path:',ir_cover_path)

            if os.path.isdir(pm_cover_path) and os.path.isdir(ir_cover_path):
                pm_file_path = os.path.join(pm_cover_path,img)
                ir_file_path = os.path.join(ir_cover_path,img)

                return pm_file_path,ir_file_path

In [ ]:
# Function to get all the files of an especific cover

patients = ['00074', '00040', '00024', '00043', '00081', '00070', '00077', '00044',
            '00002','00079', '00091', '00087', '00078', '00047', '00031', '00069',
            '00039', '00086', '00009', '00092', '00064']

covers = ['cover1','cover2','uncover']

def cover_files(path,patients,cover):
    pm_total_files = []
    ir_total_files = []
    
    for patient in patients:
        if os.path.isdir(os.path.join(path,patient)):
            print('Patient:', patient)
            patient_path = os.path.join(path,patient)
            pm_np_path = os.path.join(patient_path,'PMarray')
            ir_np_path = os.path.join(patient_path,'IRraw')

        if os.path.isdir(pm_np_path) and os.path.isdir(ir_np_path):
            pm_cover_path = os.path.join(pm_np_path,cover)
            ir_cover_path = os.path.join(ir_np_path,cover)
            filenames_pm = os.listdir(pm_cover_path)
            filenames_ir = os.listdir(ir_cover_path)

            for pm_file,ir_file in zip(filenames_pm,filenames_ir):
                pm_total_files.append(os.path.join(pm_cover_path,pm_file))
                ir_total_files.append(os.path.join(ir_cover_path,ir_file))

    return pm_total_files,ir_total_files

In [ ]:
# Function to get all the files of a patient

def patient_files(path,patient):
    pm_total_files = []
    ir_total_files = []
    
    if os.path.isdir(os.path.join(path,patient)):
        print('Patient:', patient)
        patient_path = os.path.join(path,patient)
        pm_np_path = os.path.join(patient_path,'PMarray')
        ir_np_path = os.path.join(patient_path,'IRraw')

        for category_pm,category_ir in zip(os.listdir(pm_np_path),os.listdir(ir_np_path)):
            pm_cover_path = os.path.join(pm_np_path,category_pm)
            ir_cover_path = os.path.join(ir_np_path,category_ir)
            filenames_pm = os.listdir(pm_cover_path)
            filenames_ir = os.listdir(ir_cover_path)

            for pm_file,ir_file in zip(filenames_pm,filenames_ir):
                pm_total_files.append(os.path.join(pm_cover_path,pm_file))
                ir_total_files.append(os.path.join(ir_cover_path,ir_file))

    return pm_total_files,ir_total_files


In [1]:
# class CustomDataset

class CustomDataset(Dataset):
    # Dataset for the random option
    # Includes:
    # - IR arrays
    # - PR arrays
    # Physical data
    def __init__(self, ir_paths, pm_paths, p_data, transform=None):

        self.ir_paths = ir_paths
        self.pm_paths = pm_paths
        self.p_data = p_data

        self.transform = transform

    def __len__(self):
        return len(self.ir_paths)

    def __getitem__(self, index):

        input_path = self.ir_paths[index]
        output_path = self.pm_paths[index]

        input_array = self.load_array(input_path)
        output_array = self.load_array(output_path)
        input_array = input_array.astype(np.float32)
        output_array = output_array.astype(np.float32)

        if self.transform:
            input_array = self.transform['input'](input_array)

            if PATH_DATASET == 'Server':
                parts = str(output_path.split("/")[-4])
            else:
                parts = str(output_path.split("\\")[-4])
            number = int(parts)
            p_vector = self.p_data.iloc[number-1]
            weight = p_vector[1]
            tensor_data = torch.tensor(p_vector.values)

            # Applying median filter
            median_array = signal.medfilt2d(output_array)
            max_array = np.maximum(output_array, median_array)

            area_m = 1.03226 / 10000
            ideal_pressure = weight * 9.81 / (area_m * 1000)

            output_array = (max_array / np.sum(max_array)) * ideal_pressure
            output_array = self.transform['output'](output_array)

            if USE_PHYSICAL_DATA:
                return input_array, output_array, tensor_data
            else:
                return input_array, output_array

    def load_array(self, path):
        # Load the array
        array = np.load(path)
        return array


In [2]:
# Function to get the files that we want and do the transforms and create a dataloader to pass the model

pm_files, ir_files = cover_files(LOCAL_SLP_DATASET_PATH, patients, 'cover1')

dic = {'ir': ir_files, 'pm': pm_files}

def crop_array(array):
        return crop(array,7, 29, 140, 66)

# Data transformation if needed
transform = {
        'input': transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Lambda(crop_array),  
                    transforms.Resize((192, 84)),
		            transforms.Normalize(mean=[0.5], std=[0.5]),
                    ]),
        'output': transforms.Compose([transforms.ToTensor()])}

p_data = pd.read_csv(os.path.join(LOCAL_SLP_DATASET_PATH, 'physiqueData.csv'))
p_data['gender'] = p_data['gender'].str.strip()
p_data = pd.get_dummies(p_data, columns=['gender'])
p_data = p_data.drop('sub_idx',axis=1)
p_data['gender_male'] = p_data['gender_male'].astype(int)
p_data['gender_female'] = p_data['gender_female'].astype(int)


def create_dataloader(dic,p_data,transform):
    
    dataset = CustomDataset(dic['ir'], dic['pm'], p_data, transform=transform)
    print('Len dataset:',len(dataset))

    val_loader = DataLoader(
            dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=True)
    
    val_dataset_info = {
        'Number of samples': len(val_loader.dataset),
        'Batch size': val_loader.batch_size,
        'Number of batches': len(val_loader)
        }
    
    print(f"Val loader info: {val_dataset_info}")

    return val_loader


NameError: name 'cover_files' is not defined

In [2]:
# Function to evaluate with a model loaded the files that we want with the metrics

def evaluation(model,model_file,metrics,val_loader):
    model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))
    model.to(DEVICE)

    total_metric = [0, 0]

    model.eval()

    with torch.no_grad():

        for batch_idx, (input_images, target_images) in enumerate(val_loader, 1):

            input_img= input_images.to(DEVICE)
            target_img = target_images.to(DEVICE)

            output_img = model(input_img)

            for i, metric in enumerate(metrics):

                val_metric = metric(output_img, target_img)

                total_metric[i] += val_metric

            # Free memory in each iteration
            del input_images
            del target_images
            del val_loss
            torch.cuda.empty_cache()  # Clean CUDA Cache if used GPU
            gc.collect()  # Collect trash to free memory not used

    epoch_metric = [total_metric[0] /
                    len(val_loader), total_metric[1] / len(val_loader)]
    
    print(metrics)
    print(epoch_metric)

    last_value_metrics = [0,0]

    for i, metric in enumerate(metrics):

        val_metric = metric(output_img, target_img)
        last_value_metrics[i] = val_metric

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(input_images.squeeze().cpu().numpy())
    axes[0].set_title('Input Image')

    axes[0].imshow(target_images.squeeze().cpu().numpy())
    axes[0].set_title('Target Image')

    axes[1].imshow(output_img.squeeze().cpu().numpy())
    axes[1].set_title('Output Image')

    

    fig.suptitle('Input, Target and Output Image', fontsize=12)
    fig.text(0.5, 0.01, f': {last_value_metrics:.4f}', ha='center')
    plt.savefig(os.path.join(IMG_PATH,'Comparing_output_model.png'))
    plt.show()
